# Data Prep: Overview

This notebook takes the raw data table and turns it into preprocessed csvs ready to preform ML on

The various steps before the split include data cleaning, feature engineering, and then splitting/preprocessing. 

Some of the code is taken from the EDA notebook, as I was experimenting on some of the options for this there as well. 


In [177]:
# imports
import numpy as np
import pandas as pd
import matplotlib as plt

## Data Cleaning 

The objectives here are to extract some features from one column that is a json, convert dates to datetime (to then be egineered into useful features).
Additionally, the dataset includes a warning that there might be duplicate data points, and has a likely duplicate field that needs to be adressed to prevent data leakage. (multiple reports about the same breach). These are removed. 

In [194]:
# Load the data
path = "/Users/djfiume/Desktop/DSI/1030/data-breach-ml"
breaches = pd.read_csv(path + "/data/PRC-DataBreachChronology-v1.5-01.31.2024-NOBREACHNOTIFICATIONLETTERS.csv")
print("The (rows, columns) of this dataset is: \n ", breaches.shape)
#breaches.head()

The (rows, columns) of this dataset is: 
  (35167, 28)


In [195]:
# First drop rows that do not contain information about our target variable

breaches["Max Records Impacted"] = breaches["Max Records Impacted"].replace("UNKN", None)
breaches["Max Records Impacted"] = breaches["Max Records Impacted"].apply(pd.to_numeric)
breaches = breaches.dropna(subset = ["Max Records Impacted"], axis="index")


print("The (rows, columns) of this dataset is: \n ", breaches.shape)

The (rows, columns) of this dataset is: 
  (27044, 28)


In [196]:
# Next, we deal with the duplicates - if its the same company with the same number of records breached
# on the same date, we will remove it. The best way to do this would be to take the two rows and make a new one with all the 
# combined data, but I did not end up implementing that. 

# The reason we are doing this in the first place is to prevent data leackage. 

breaches = breaches.drop_duplicates(subset=['Name of Entity', "Max Records Impacted", "Date of Breach"]
                                    ,keep="first")

# Reindex here to prevent problems down the road 
breaches = breaches.reset_index(drop=True)

print("The (rows, columns) of this dataset is: \n ", breaches.shape)

The (rows, columns) of this dataset is: 
  (24586, 28)


In [197]:
# We narrow down the features we are interestd in (to see all features, go to EDA notebook)
interested_cols = ["Information Types", "Breach Type", "Source", "Organization Type", "breach_location_state", "Reported Date", "Date of Breach", "Date of Breach End", "Max Records Impacted"]
breaches = breaches[interested_cols]
print("The (rows, columns) of this dataset is: \n ", breaches.shape)

The (rows, columns) of this dataset is: 
  (24586, 9)


In [198]:
# The Information Types Column has json data we want: the encryption status of the data as well as the type of data breached. We must extract it
import ast

# Turn the column into a list of dictionaries
print(breaches["Information Types"][1])
breaches["Information Types"] = breaches["Information Types"].apply(lambda x: x.strip())
breaches["Information Types"] = breaches["Information Types"].apply(ast.literal_eval)
print(breaches["Information Types"][1])

print("The (rows, columns) of this dataset is: \n ", breaches.shape)


[{"type": "UNKN", "encryption_status": "UNKN", "explanation": "No specific information about the types of information exposed in the breach is provided."}]
[{'type': 'UNKN', 'encryption_status': 'UNKN', 'explanation': 'No specific information about the types of information exposed in the breach is provided.'}]
The (rows, columns) of this dataset is: 
  (24586, 9)


## Feature Engineering

I engineer two types of features. First, I extract each type of data from the information types json/col - each data breach may contain multiple types of data and encryption statuses. I will create columns for each type and count the number of instances for each breach. 

Secondly, I am engineering date features. In essense, I am converting the date features into those that can be used by a machine learning model. The date of breach will be present in the form of a "days since first breach in the dataset" kind of thing, and the other date features will be represented in the length of time it took to report the breach, as well as the length of the breach (utilizing the dates of end of breach and date reported features.)

In [199]:
# Extract info from Information Types Column
info = pd.json_normalize(breaches["Information Types"])
print("The (rows, columns) of the expaned information column is: \n ", info.shape)
info.head()

new_info = pd.json_normalize(info[0])
new_info = new_info[["type", "encryption_status"]]
new_info.head()

The (rows, columns) of the expaned information column is: 
  (24586, 7)


,type,encryption_status
0,IDENTIFIER,UNENCRYPTED
1,UNKN,UNKN
2,UNKN,UNKN
3,UNKN,UNKN
4,UNKN,UNENCRYPTED


In [203]:
# Each column here represents the nth entry (type of data in the breach). Unpack each one separately and add to the table
new_info = pd.json_normalize(info[0])
new_info_type = pd.DataFrame(new_info["type"])
new_info_encrypt = pd.DataFrame(new_info["encryption_status"])

for i in range(1, info.shape[1]):

    info_i = pd.json_normalize(info[i])
    
    info_i = info_i[["type", "encryption_status"]]
    info_i.columns = ["type" + str(i), "encryption_status" + str(i)]
    new_info_type = new_info_type.join(info_i["type" + str(i)])
    new_info_encrypt = new_info_encrypt.join(info_i["encryption_status" + str(i)])
    #print(new_info.head())

new_info_type["type_count"] = new_info_type.count(axis=1)
new_info_type.head(10)
# breaches = breaches.join(info)
# print("The (rows, columns) of this dataset is: \n ", breaches.shape)
# breaches.head()

#info = info.explode("Information Types", ignore_index=True)


,type,type1,type2,type3,type4,type5,type6,type_count
0,IDENTIFIER,SENSITIVE-GOV,IDENTIFIER,SENSITIVE-LOGIN,EMPLOYMENT,NaN,NaN,5
1,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,UNKN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,IDENTIFIER,HEALTH,SENSITIVE-GOV,NaN,NaN,NaN,NaN,3
9,IDENTIFIER,EMPLOYMENT,SENSITIVE-LOGIN,NaN,NaN,NaN,NaN,3


In [204]:
new_info_encrypt.head(10)


,encryption_status,encryption_status1,encryption_status2,encryption_status3,encryption_status4,encryption_status5,encryption_status6
0,UNENCRYPTED,UNENCRYPTED,UNENCRYPTED,UNENCRYPTED,UNENCRYPTED,NaN,NaN
1,UNKN,NaN,NaN,NaN,NaN,NaN,NaN
2,UNKN,NaN,NaN,NaN,NaN,NaN,NaN
3,UNKN,NaN,NaN,NaN,NaN,NaN,NaN
4,UNENCRYPTED,NaN,NaN,NaN,NaN,NaN,NaN
5,UNKN,NaN,NaN,NaN,NaN,NaN,NaN
6,UNKN,NaN,NaN,NaN,NaN,NaN,NaN
7,UNKN,NaN,NaN,NaN,NaN,NaN,NaN
8,UNENCRYPTED,UNENCRYPTED,UNENCRYPTED,NaN,NaN,NaN,NaN
9,UNENCRYPTED,UNENCRYPTED,UNENCRYPTED,NaN,NaN,NaN,NaN


In [212]:
# The easy thing to do is just take the first column and have some information, but I will basically count how many pieces of information are of each type - i.e.
# does the breach contain each information type or encryption type (they can contain multiple) and how much

info_types = ["IDENTIFIER", "COMMERCIAL", "BIOMETRIC", "HEALTH", "INTERNETDATA", "GEOLOCATION", "RECORDING", "EMPLOYMENT", "EDUCATION", "SENSITIVE-GOV", "SENSITIVE-LOGIN", "SENSITIVE-GEOLOCATION", "SENSITIVE-PROTECTED", "SENSITIVE-COMMUNICATIONS", "SENSITIVE-DNA", "UNKN"]

# counts the strings (must exaxt match)
def count_strings_row(row, string_list):
    return {string: row.str.count(r'\b' + string + r'\b').sum() for string in string_list}

# apply the function to each row of the dataframe
counts_type = new_info_type.apply(lambda row: count_strings_row(row, info_types), axis=1)
counts_type = pd.DataFrame(counts_type.tolist())

counts_type.head()

,IDENTIFIER,COMMERCIAL,BIOMETRIC,HEALTH,INTERNETDATA,GEOLOCATION,RECORDING,EMPLOYMENT,EDUCATION,SENSITIVE-GOV,SENSITIVE-LOGIN,SENSITIVE-GEOLOCATION,SENSITIVE-PROTECTED,SENSITIVE-COMMUNICATIONS,SENSITIVE-DNA,UNKN
0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [213]:
# Do the same with encrypted 
encrypt_types = ["ENCRYPTED", "ENCRYPTED-WITH-DECRYPTIONKEY", "UNENCRYPTED", "UNKN"]
counts_encrypt = new_info_encrypt.apply(lambda row: count_strings_row(row, encrypt_types), axis=1)
counts_encrypt = pd.DataFrame(counts_encrypt.tolist())
counts_encrypt.head()

,ENCRYPTED,ENCRYPTED-WITH-DECRYPTIONKEY,UNENCRYPTED,UNKN
0,0.0,0.0,5.0,0.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0


In [226]:
# Look at this data since its interesting (about half unkn, but still interesting)
print(counts_type.describe())
print(counts_encrypt.describe())

print(counts_type.shape)
print(counts_encrypt.shape)

# Add to the data
counts_type = counts_type.rename(columns={"UNKN": "Type UNKN"})
counts_encrypt = counts_encrypt.rename(columns={"UNKN": "Encrypt UNKN"})
# breaches = pd.concat([breaches, counts_encrypt, counts_encrypt])

print("The (rows, columns) of this dataset is: \n ", breaches.shape)


         IDENTIFIER    COMMERCIAL     BIOMETRIC        HEALTH  INTERNETDATA  \
count  24586.000000  24586.000000  24586.000000  24586.000000  24586.000000   
mean       0.463475      0.098633      0.000691      0.169568      0.000529   
std        0.524747      0.298448      0.026287      0.375694      0.022989   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        1.000000      0.000000      0.000000      0.000000      0.000000   
max        4.000000      2.000000      1.000000      2.000000      1.000000   

        GEOLOCATION     RECORDING    EMPLOYMENT     EDUCATION  SENSITIVE-GOV  \
count  24586.000000  24586.000000  24586.000000  24586.000000   24586.000000   
mean       0.001424      0.000651      0.042951      0.002440       0.271659   
std        0.037704      0.025503      0.202751 

Now for the time features

In [228]:
# Look at Date Time Features
# Reported Date
# Date of Breach
# Date of Breach End

dates = breaches[["Reported Date", "Date of Breach", "Date of Breach End"]]
dates = dates.replace("UNKN", None)
# Bad Data
dates = dates.replace("2014/00/00", None)
dates = dates.replace("2015/09/31", None)
dates = dates.replace("2016/02/31", None)
dates = dates.replace("1997/06/02", None)
dates = dates.replace("1999/09/22", None)
dates = dates.replace("2000/01/01", None)

# dates = dates.dropna(axis="index")
dates[["Reported Date", "Date of Breach", "Date of Breach End"]] = (
     dates[["Reported Date", "Date of Breach", "Date of Breach End"]].apply(pd.to_datetime, format='mixed'))
dates["Length of Breach (Days)"] = (dates["Date of Breach End"] - dates["Date of Breach"]).dt.days.clip(lower=0)
dates["Days Until Reported"] = (dates["Reported Date"] - dates["Date of Breach"]).dt.days.clip(lower=0)

early = min(dates["Date of Breach"])
print(early)
print(early.year)

# # Columns for Year/Month
dates["Reported Year"] = dates["Reported Date"].dt.year
dates["Reported Month"] = dates["Reported Date"].dt.month

print(dates.describe())
dates

2000-01-01 00:00:00
2000
                       Reported Date                 Date of Breach  \
count                          24310                          18878   
mean   2018-05-30 14:56:16.980666368  2018-02-06 17:58:53.637037824   
min              2006-11-10 00:00:00            2000-01-01 00:00:00   
25%              2016-09-08 00:00:00            2016-06-17 00:00:00   
50%              2018-07-30 00:00:00            2018-03-05 00:00:00   
75%              2020-06-18 00:00:00            2019-11-27 00:00:00   
max              2023-09-28 00:00:00            2023-11-15 00:00:00   
std                              NaN                            NaN   

                  Date of Breach End  Length of Breach (Days)  \
count                           6451              6283.000000   
mean   2017-08-20 21:10:47.899550208               131.499284   
min              2006-12-01 00:00:00                 0.000000   
25%              2016-08-01 12:00:00                 6.000000   
50%       

,Reported Date,Date of Breach,Date of Breach End,Length of Breach (Days),Days Until Reported,Reported Year,Reported Month
0,2017-04-28,2017-03-30,NaT,NaN,29.0,2017.0,4.0
1,2022-09-28,2022-05-19,NaT,NaN,132.0,2022.0,9.0
2,2018-02-13,2017-11-21,2017-12-08,17.0,84.0,2018.0,2.0
3,2023-06-02,2023-01-05,NaT,NaN,148.0,2023.0,6.0
4,2017-08-04,2017-02-16,2017-06-25,129.0,169.0,2017.0,8.0
...,...,...,...,...,...,...,...
24581,2018-04-19,2018-03-06,NaT,NaN,44.0,2018.0,4.0
24582,2017-03-27,2017-02-10,NaT,NaN,45.0,2017.0,3.0
24583,2018-07-26,2018-02-18,2018-04-28,69.0,158.0,2018.0,7.0
24584,2019-07-15,2019-05-03,2019-05-06,3.0,73.0,2019.0,7.0


## Export Data

This is just selecting the data that will be used in the model. You can select additional features if you want! The actual numerical preprocessing will be done in the ML Pipeline to account for randomness in splitting / etc. 

In [234]:
# Putting it all together

breach_cols = ["Max Records Impacted", "Breach Type", "Source", "Organization Type", "breach_location_state"]
breach = breaches[breach_cols]

date_cols = ["Reported Year", "Reported Month", "Days Until Reported", "Length of Breach (Days)"]
date = dates[date_cols]

print(breach.shape)
print(date.shape)
print(counts_type.shape)
print(counts_encrypt.shape)

cleaned_raw_data = pd.concat([breach,date,counts_type,counts_encrypt], axis=1)



print("Cleaned Raw Data Shape: ", cleaned_raw_data.shape)
cleaned_raw_data.head()


(24586, 5)
(24586, 4)
(24586, 16)
(24586, 4)
Cleaned Raw Data Shape:  (24586, 29)


,Max Records Impacted,Breach Type,Source,Organization Type,breach_location_state,Reported Year,Reported Month,Days Until Reported,Length of Breach (Days),IDENTIFIER,...,SENSITIVE-LOGIN,SENSITIVE-GEOLOCATION,SENSITIVE-PROTECTED,SENSITIVE-COMMUNICATIONS,SENSITIVE-DNA,Type UNKN,ENCRYPTED,ENCRYPTED-WITH-DECRYPTIONKEY,UNENCRYPTED,Encrypt UNKN
0,18100.0,HACK,IA,BSO,Virginia,2017.0,4.0,29.0,NaN,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,14599.0,UNKN,IN,GOV,UNKN,2022.0,9.0,132.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1.0,UNKN,ME,BSR,New Jersey,2018.0,2.0,84.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1085.0,UNKN,IN,BSO,Indiana,2023.0,6.0,148.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,UNKN,ME,BSO,UNKN,2017.0,8.0,169.0,129.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [235]:
cleaned_raw_data.to_csv(path + "/data/cleaned_raw_data.csv")